In [ ]:
%pip install --upgrade pip 
%pip install -U langchain  
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph  
%pip install -qU "langchain[mistralai]" 
%pip install -qU langchain-huggingface
%pip install sentence-transformers
%pip install -qU langchain-core
%pip install -qU langchain_community pypdf pillow
%pip install hf_xet

INDEXING

In [36]:
import getpass
import os
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from typing import NamedTuple, Tuple
from langgraph.graph import START, StateGraph
from dotenv import load_dotenv,find_dotenv

In [37]:
# langsmith e os e getpass para lidar com as chaves de api
load_dotenv(find_dotenv())

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key fot langsmith: ")

Enter API key fot langsmith:  ········


In [38]:
# chat model
if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

llm = init_chat_model("mistral-small-2503", model_provider = "mistralai")

In [39]:
# embedding model
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

In [40]:
# vector in-memory
vector_store = InMemoryVectorStore(embeddings)

In [41]:
# Loading documents em pdf
directory_path = (
    "C:/JupyterNotebook/RAG/RAG_exames"
)
loader = PyPDFDirectoryLoader("RAG_exames/")

docs = loader.load()

In [42]:
# Text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

In [43]:
# Storing documents
document_ids = vector_store.add_documents(documents=all_splits)

INICIO DO RETRIEVAL E GENERATION

In [44]:
# prompt
# fazer teste em txt
template = """Você é um especialista em análise de relatórios de mamografia. Por favor, leia o relatório quando indicado e extraia as seguintes informações:
Cisto:
- Presente ou Ausente
- Localização e tamanho do cisto
Nódulo:
- Presente ou Ausente
- Localização e tamanho do nódulo
Calcificação:
- Presente ou Ausente
- Localização e tamanho da calcificação
Microcalcificação:
- Presente ou Ausente
- Localização e tamanho da microcalcificação
BI-RADS: [valor]
Outras citações a avaliar: [observações adicionais relevantes]

Caso não encontre alguma informação que se encaixe, coloque [sem referência no texto].

Diretrizes de Interpretação

Diferenciação entre Nódulo e Cisto:

Se um achado é identificado inicialmente como "nódulo" na mamografia, mas confirmado como "cisto" no ultrassom, classifique apenas como CISTO (presente).
Nódulos são estruturas sólidas; cistos são estruturas predominantemente líquidas.
Complexos sólido-císticos devem ser reportados em ambas categorias (nódulo E cisto).


Priorização de Achados Múltiplos:

Quando houver múltiplos cistos/nódulos, reporte TODOS, priorizando:
a) Achados classificados como suspeitos pelo relatório
b) Achados de maior tamanho
c) Achados com características atípicas mencionadas


Diferenciação entre Calcificações e Microcalcificações:

Calcificações: estruturas maiores, geralmente descritas como "grosseiras", "distróficas", "vasculares"
Microcalcificações: estruturas menores, frequentemente descritas como "puntiformes", "pleomórficas", "lineares", "agrupadas", "em cluster"
Se o relatório mencionar "microcalcificações", classifique especificamente como microcalcificações
Se mencionar apenas "calcificações", classifique como calcificações.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [45]:
# estado para o langgraph
class State(NamedTuple):
    question: str
    context: Tuple[Document, ...]
    answer: str

In [49]:
# Funções do rag
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state.question)
    return State(
        question=state.question, 
        context=tuple(retrieved_docs), 
        answer=state.answer
    )

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state.context)
    messages = custom_rag_prompt.invoke({"question": state.question, "context": docs_content})
    response = llm.invoke(messages)
    return State(
        question=state.question,
        context=state.context,
        answer=response.content
    )

In [50]:
# langgraph
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [53]:
state_inicial = State(
    question="""Faça a extração das característica do seguinte exame mamográfico.

MAMOGRAFIA DIGITAL DR* BILATERAL

Indicação clínica: 69 anos. Rotina. Antecedente de neoplasia mamária.

Exame com MAMÓGRAFO DIGITAL nas incidências craniocaudal e mediolateral oblíqua acrescido de incidências em ambas projeções obtidas com manobras de deslocamento posterior dos implantes mamários.

Status pós cirurgia conservadora da mama esquerda.
Parênquima mamário heterogeneamente denso, o que reduz a sensibilidade da mamografia.
Alterações arquiteturais, relacionadas à mamoplastia.
Nódulo denso de contornos espiculados projetado no QSE da mama esquerda, associado a retração cutânea,
com correspondência ao ultrassom, maior em relação ao exame de 01/2024. Prosseguir com core biopsy.
Cisto oleoso na mama esquerda.
Calcificações esparsas.
Ausência de microcalcificações pleomórficas agrupadas ou ramificadas.
Implante bilateral, sem sinais de roturas extracapsulares.
Linfonodo axilar, de aspecto reacional.

ACR-BIRADS® categoria 5.""",
    context=tuple([]),
    answer=" "
)

response = graph.invoke(state_inicial)
print(response["answer"])

### Extração das Características do Exame Mamográfico

**Cisto:**
- **Presente ou Ausente:** Presente
- **Localização e tamanho do cisto:** Cisto oleoso na mama esquerda (tamanho não especificado)

**Nódulo:**
- **Presente ou Ausente:** Presente
- **Localização e tamanho do nódulo:** Nódulo denso de contornos espiculados projetado no quadrante súpero-externo (QSE) da mama esquerda (tamanho não especificado)

**Calcificação:**
- **Presente ou Ausente:** Presente
- **Localização e tamanho da calcificação:** Calcificações esparsas (localização e tamanho não especificados)

**Microcalcificação:**
- **Presente ou Ausente:** Ausente
- **Localização e tamanho da microcalcificação:** [sem referência no texto]

**BI-RADS:** 5

**Outras citações a avaliar:**
- Alterações arquiteturais relacionadas à mamoplastia.
- Retração cutânea associada ao nódulo.
- Implante bilateral, sem sinais de roturas extracapsulares.
- Linfonodo axilar de aspecto reacional.
- Status pós cirurgia conservadora da mama e

In [54]:
# langraph acompanhando o que o RAG usou de documentos
print(f'Context: {response["context"]}\n\n')

Context: (Document(id='7837dcb1-c133-44b4-ab25-ff8fa4b8e61e', metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2025-06-28T19:39:27-03:00', 'author': 'Hanna Reckziegel', 'moddate': '2025-06-28T19:39:27-03:00', 'source': 'RAG_exames\\laudo_2_extraido.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'start_index': 0}, page_content='MAMOGRAFIA DIGITAL (DR) E ECOGRAFIA MAMÁRIA \nMamas simétricas constituídas predominantemente por tecido fibroglandular. \nAusência de microcalcificações suspeitas nas mamas. \nNo presente exame não há evidência de imagem com aspecto de nódulo ou \ncisto nas mamas. \nNão se observa imagem sugestiva de neoplasia. \nLinfonodos localizados nos prolongamentos axilares das mamas. \nA comparação com o exame anterior datado de 16.01.2017 demonstra não ter \ncorrido alteração significativa. \n \nBIRADS - 1 (exame normais). \n \nEXTRAÇÃO: \n \nCisto:  \n- Presente ou Ausente: ausente \n- Localização do cisto e tamanh

In [ ]:
for step in graph.stream(
    {"question": """Faça a extração das característica do seguinte exame mamográfico.

MAMOGRAFIA DIGITAL DR* BILATERAL

Indicação clínica: 69 anos. Rotina. Antecedente de neoplasia mamária.

Exame com MAMÓGRAFO DIGITAL nas incidências craniocaudal e mediolateral oblíqua acrescido de incidências em ambas projeções obtidas com manobras de deslocamento posterior dos implantes mamários.

Status pós cirurgia conservadora da mama esquerda.
Parênquima mamário heterogeneamente denso, o que reduz a sensibilidade da mamografia.
Alterações arquiteturais, relacionadas à mamoplastia.
Nódulo denso de contornos espiculados projetado no QSE da mama esquerda, associado a retração cutânea,
com correspondência ao ultrassom, maior em relação ao exame de 01/2024. Prosseguir com core biopsy.
Cisto oleoso na mama esquerda.
Calcificações esparsas.
Ausência de microcalcificações pleomórficas agrupadas ou ramificadas.
Implante bilateral, sem sinais de roturas extracapsulares.
Linfonodo axilar, de aspecto reacional.

ACR-BIRADS® categoria 5."""}, stream_mode="updates"
):
    print(f"{step}\n\n----------------\n")